In [1]:
from ngboost import NGBRegressor
from scipy.stats import expon
from scipy.stats import multivariate_normal
from sklearn.datasets import load_boston
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from ngboost.scores import *
from ngboost.distns import *
from pandas import read_csv
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
import pickle
from sklearn.tree import DecisionTreeRegressor

def backup(object, filename):
    with open(filename, 'wb') as f:
        pickle.dump(object, f, pickle.HIGHEST_PROTOCOL)

def reload(persist):
    with open(persist, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        data = pickle.load(f)
    return data

# series = read_csv('ProcessedDataCenterHall.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
#series = read_csv('ProcessedBioEngineeringPV.csv', header=0, parse_dates=True)
outputs_feature_select = reload("outputs_feature_select.pickle")



In [2]:
dataset = pd.read_csv("ngboost_dataset.csv.gz")

In [3]:
dataset.set_index("DateTime",inplace=True)


In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset.iloc[:,:-24*2], dataset.iloc[:,-24*2+6:-24*2+8], test_size=0.2, shuffle=False)
# X_train, X_test, Y_train, Y_test = train_test_split(x_series_value, y_date_numerical, test_size=0.2)

In [5]:
ngb = NGBRegressor(Dist=MultivariateNormal(k=2), Score=LogScore, minibatch_frac=0.2, n_estimators=400, verbose=True).fit(X_train, Y_train, X_val=X_test, Y_val=Y_test)
backup(ngb,"ngb_singstep.pickle")
# ngb = NGBRegressor(Dist=Normal, Score=LogScore, minibatch_frac=0.4, n_estimators=1000, verbose=True).fit(X_train, Y_train, X_val=X_test, Y_val=Y_test)
# backup(ngb,"ngb_singstep.pickle")
 

[iter 0] loss=8.3449 val_loss=8.3441 scale=1.0000 norm=41.1096
[iter 100] loss=7.2800 val_loss=7.2899 scale=1.0000 norm=18.1055
[iter 200] loss=6.6758 val_loss=6.6870 scale=1.0000 norm=10.2629
[iter 300] loss=6.2203 val_loss=6.2362 scale=1.0000 norm=7.7470


In [6]:
%%time
Y_preds = ngb.predict(X_test[0:2])
Y_dists = ngb.pred_dist(X_test[0:2])


CPU times: user 238 ms, sys: 10 ms, total: 248 ms
Wall time: 245 ms


In [7]:
Y_dists[0:2].params

{'loc': array([[90.08248943, 53.76808253],
        [81.00534693, 40.79780645]]),
 'scale': array([[[168.10238018,  70.09904413],
         [ 70.09904413,  40.12289435]],
 
        [[192.75418376,  80.29570341],
         [ 80.29570341,  38.48529688]]])}

In [8]:
Y_test[:2]

,realpower_15min_3,reactivepower_15min_3
DateTime,,
2017-09-21 22:45:00,76.810,48.170
2019-09-27 04:45:00,92.944,55.001


In [7]:
dataset.iloc[:,-24*2+6:-24*2+8]

,realpower_15min_3,reactivepower_15min_3
DateTime,,
2019-01-02 19:00:00,113.203,57.206
2017-03-19 09:30:00,114.789,66.486
2017-06-12 18:30:00,144.979,63.804
2017-12-24 20:45:00,75.667,36.607
2016-07-28 16:00:00,151.912,71.614
...,...,...
2018-09-11 08:15:00,148.556,67.318
2017-08-10 15:45:00,180.233,88.278
2018-06-23 20:00:00,73.946,40.380


In [9]:
# Deterministic Values
print("Y_preds: ", Y_preds)
print("Y_dist: ", Y_dists)

# Probabilistic Values
mean = Y_dists.mean
sample = Y_dists.rv()
scipy_list = Y_dists.scipy_distribution()

print("mean: ", mean)
print("sample: ", sample)

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, Y_test)
print('Test MSE', test_MSE)

# test Negative Log Likelihood
test_NLL = -Y_dists.logpdf(Y_test).mean()
print('Test NLL', test_NLL)


Y_preds:  [[113.04805474  55.3140447  113.07356297 ...  54.89645608 113.12103608
   54.98728074]
 [117.27310087  56.9993305  117.32186529 ...  57.47288317 119.45798949
   57.35950477]
 [113.37981554  55.24064138 113.24663968 ...  55.20674841 112.97813482
   55.18324966]
 ...
 [113.39605679  55.31512795 113.74547368 ...  55.33690686 113.26293509
   55.26456663]
 [117.21523261  56.81931229 117.09137051 ...  54.84898354 112.98750505
   54.79387924]
 [112.97509098  54.6257698  112.89424341 ...  55.25544963 113.33831528
   55.24761092]]
Y_dist:  <ngboost.distns.multivariate_normal.MultivariateNormal.<locals>.MVN object at 0x7fa0846be950>
mean:  <bound method MultivariateNormal.<locals>.MVN.mean of <ngboost.distns.multivariate_normal.MultivariateNormal.<locals>.MVN object at 0x7fa0846be950>>
sample:  [[ 84.24356859  44.11937465  82.97863548 ...  54.16785351 111.70072369
   53.95960561]
 [ 61.27140517  31.4686588   63.03235412 ...  46.63761491  99.46107505
   44.01034543]
 [147.66620238  78.5

In [10]:
dataset

,realpowerin_3Mos_0,reactivepowerin_3Mos_0,realpowerin_3Mos_1,reactivepowerin_3Mos_1,realpowerin_3Mos_2,reactivepowerin_3Mos_2,realpowerin_3Mos_3,reactivepowerin_3Mos_3,realpowerin_3Mos_4,reactivepowerin_3Mos_4,...,realpower_15min_19,reactivepower_15min_19,realpower_15min_20,reactivepower_15min_20,realpower_15min_21,reactivepower_15min_21,realpower_15min_22,reactivepower_15min_22,realpower_15min_23,reactivepower_15min_23
DateTime,,,,,,,,,,,,,,,,,,,,,
2019-01-02 19:00:00,69.576594,40.223632,98.061426,48.441935,106.654635,51.540939,116.407210,55.778000,109.860289,53.896198,...,77.642,42.401,79.550,42.490,76.909,41.283,76.836,41.192,74.102,39.839
2017-03-19 09:30:00,132.074315,63.896061,129.718024,60.926313,127.623560,61.582185,119.290911,59.863786,122.219307,59.179844,...,136.384,73.642,138.695,74.732,137.149,74.826,138.237,75.738,140.604,77.189
2017-06-12 18:30:00,131.987805,57.367921,119.023399,51.382314,129.978625,57.416091,133.018625,58.082524,130.768388,58.410141,...,98.690,60.109,98.859,58.981,95.725,56.766,96.065,59.261,88.002,54.556
2017-12-24 20:45:00,105.168007,55.564362,120.675549,62.098315,123.220878,62.464774,123.615202,62.449284,120.430170,61.577256,...,66.565,33.464,66.726,34.035,65.146,33.680,66.730,33.719,65.692,33.701
2016-07-28 16:00:00,108.900911,51.716554,103.149996,48.107582,100.738131,47.053076,105.031512,48.938476,98.947372,46.861061,...,121.376,52.454,117.741,51.409,117.166,51.087,116.149,50.779,119.001,52.907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-09-11 08:15:00,118.874863,62.886286,119.129723,61.691338,124.814207,62.519354,127.158900,63.604836,130.362058,63.587170,...,175.242,79.400,175.202,79.391,174.774,78.322,175.476,78.323,177.478,80.050
2017-08-10 15:45:00,135.679872,68.503467,123.970068,61.531984,122.488994,62.611262,125.579586,62.162750,114.901070,60.860920,...,129.125,63.058,124.402,61.010,123.453,61.863,125.022,62.501,122.569,60.733
2018-06-23 20:00:00,113.123411,56.950832,118.542074,56.523092,107.797534,53.785341,112.394710,56.385756,117.524812,60.311214,...,74.253,44.458,74.407,43.882,74.642,43.947,73.781,43.757,74.259,43.661
